In [2]:
# ! gsutil -m cp gs://noobs-ml/sem-search/papers_set_with_restored_abstract.csv .
# ! gsutil -m cp gs://noobs-ml/sem-search/top_100_authors.csv .

In [4]:
# ! pip install sentence-transformers

In [1]:
import pandas as pd
import os
import gc
import ast
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.spatial
import json
import re
import numpy as np

In [52]:
full_df = pd.read_csv("papers_set_with_restored_abstract.csv")

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (2,7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [53]:
full_df.fillna("None-placeholder", inplace=True)

In [54]:
full_df.drop(["Unnamed: 0", "Unnamed: 0.1", "page_end", "page_start", "publisher", "issue", "indexed_abstract", 
              "volume", "venue", "doc_type"], inplace=True, axis=1)

In [4]:
full_df.abstract.iloc[1]

'Background\r\nEnvironmental sequence datasets are increasing at an exponential rate; however, the vast majority of them lack appropriate descriptors like sampling location, time and depth/altitude: generally referred to as metadata or contextual data. The consistent capture and structured submission of these data is crucial for integrated data analysis and ecosystems modeling. The application MetaBar has been developed, to support consistent contextual data acquisition.'

In [5]:
embedder = SentenceTransformer('roberta-base-nli-stsb-mean-tokens')

In [6]:
## Embedding speed experiments

In [38]:
test_texts = ["Environmental sequence datasets are increasing at an exponential rate; \
              however, the vast majority of them lack appropriate descriptors like sampling location, time and depth/altitude: generally \
              referred to as metadata or contextual data. The consistent capture and structured submission of these data \
              is crucial for integrated data analysis and ecosystems modeling. \
              The application MetaBar has been developed, to support consistent contextual data acquisition."]*10000

In [39]:
split_test_texts = [[el for el in t.split('.') if el != ''] for t in test_texts]

In [40]:
split_test_texts[0]

['Environmental sequence datasets are increasing at an exponential rate;               however, the vast majority of them lack appropriate descriptors like sampling location, time and depth/altitude: generally               referred to as metadata or contextual data',
 ' The consistent capture and structured submission of these data               is crucial for integrated data analysis and ecosystems modeling',
 '               The application MetaBar has been developed, to support consistent contextual data acquisition']

In [43]:
%%time
encoded_test_texts = [get_average_sentences_embedding(s) for s in split_test_texts]

CPU times: user 1min 51s, sys: 228 ms, total: 1min 51s
Wall time: 1min 51s


In [30]:
# --- Batch encoding goes very fast, around 14 seconds for 10.000 sentences. 
# So it means that if we assume our document is around 5 sentences, and we have 300.000 documents, that is then 1.500.000 sentences.
# 1.500.000 / 30.000 = 150 ==> 150 * 14 seconds ==> 2100 seconds ==> 35 minutes
# I would just need to reformat the way it's fed in...

In [2]:
full_df

NameError: name 'full_df' is not defined

In [42]:
def get_average_sentences_embedding(sentences, model=embedder):
    embeddings = model.encode(sentences)
    return np.mean(embeddings, axis=0)

def calculate_average_embedding_abstract_and_title(abstract, title, strategy="merge", model=embedder):

    title_embedding = model.encode(title)
    
    if strategy == "merge":
        abstract_embeddings = model.encode(abstract)
        merged = np.concatenate((title_embedding, abstract_embeddings), axis=0)
        average = np.mean(merged, axis=0)
    elif strategy == "separate":
        average_abstract = get_average_sentences_embedding(abstract)
        average = np.mean([average_abstract, title_embedding[0]], axis=0)
    else:
        print("Warning: wrong strategy is used. Use either 'merge' or 'separate'. Proceeding using the 'merge' strategy.")
        abstract_embeddings = model.encode(abstract)
        merged = np.concatenate((title_embedding, abstract_embeddings), axis=0)
        average = np.mean(merged, axis=0)
        
    return average

In [30]:
def create_embedding(row, strat="merge"):
    abstract = row.abstract
    title = row.title
    if abstract and abstract != "None-placeholder":
        if title:
            # ... get embedding based on abstract and title
            emb = calculate_average_embedding_abstract_and_title(abstract,
                                                                  title,
                                                                  strat)
            return emb
        else:
            # ... get embedding just based on abstract
            # Feed preprocessed clean sentences into this. For now, just use split() on text. 

            emb = get_average_sentences_embedding(abstract.split())
            return emb
    else:
        if title and title != "None-placeholder":
            # ... get embedding based on just title
            emb = get_average_sentences_embedding([title])
            return emb
        else:
            # ... no embedding possible, ignore the paper
            return []

In [ ]:
full_df["sbert_merged_embedding"] = full_df.apply(lambda x: create_embedding(x), axis=1)

In [ ]:
full_df.to_csv("new_data.csv")

In [1]:
2

2

In [ ]:
full_df["sbert_separate_embedding"] = full_df.apply(lambda x: create_embedding(x, "separate"), axis=1)

In [ ]:
full_df.to_csv("new_data_with_separate_and_merged.csv")

In [1]:
# code to embed all the titles
title_embeddings = model.encode(full_df.title)